In [1]:
import pandas as pd
import joblib
import os
import time
import threading
from dotenv import load_dotenv

# Plotly (차트)
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ipywidgets (UI)
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout
from IPython.display import display, clear_output

print("라이브러리 로드 완료.")

라이브러리 로드 완료.


In [2]:
# .env 파일 로드 (notebooks/ -> ../.env)
load_dotenv('../.env')

DATA_PATH = os.getenv('DATA_PATH')
MODEL_PATH = os.getenv('MODEL_PATH')

# 보고서 5.1장 기반 기준값
OPTIMAL_VALUES = {
    'EX1.Z1_PV': 210.12,
    'EX1.MELT_P_PV': 6.86,
    'EX1.MD_PV': 71.32
}
# 그래프에 표시할 주요 파라미터
MAIN_PARAMETERS = ['EX1.Z1_PV', 'EX1.MELT_P_PV', 'EX1.MD_PV']

# 그래프용 데이터 저장소
time_points = []
param_z1_data = []
param_melt_p_data = []
param_md_data = []
log_data = [] # 테이블 로그용
MAX_GRAPH_POINTS = 200 # 그래프에 표시할 최대 데이터 포인트 수

print(f"데이터 경로: {DATA_PATH}")
print(f"모델 경로: {MODEL_PATH}")

데이터 경로: ../data/kamp_main_data.csv
모델 경로: ../models/ml_core_pipeline.joblib


In [3]:
import pandas as pd
import joblib
import os
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

print("환경 변수 확인...")
if not DATA_PATH or not MODEL_PATH:
    print("오류: .env 파일에서 DATA_PATH 또는 MODEL_PATH를 찾을 수 없습니다.")
else:
    try:
        # 1. 데이터 로드
        print("CSV 데이터 로딩 중...")
        df = pd.read_csv(DATA_PATH)

        # 2. 데이터 전처리
        df.dropna(subset=['passorfail'], inplace=True)
        df['passorfail'] = df['passorfail'].astype(int)

        # 3. (핵심) 훈련 데이터 분리
        # TQ=72인 데이터만 필터링하여 훈련 세트 구성
        print("TQ=72인 데이터만 필터링하여 훈련 세트 구성 중...")
        df_for_training = df[df['EX1.MD_TQ'] == 72].copy()
        
        # 4. 특성(Features) 및 타겟(Target) 정의
        target = 'passorfail'
        features = [
            col for col in df.columns 
            if col not in ['date', 'passorfail', 'EX1.MD_TQ']
        ]

        X_train = df_for_training[features]
        y_train = df_for_training[target]
        
        print(f"총 {len(df_for_training)}건의 데이터로 모델 학습 시작...")
        print(f"학습될 불량(Fail) 샘플 수: {y_train.sum()} (8건이어야 함)")

        # 5. 모델 및 스케일러 학습
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)

        model = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)
        model.fit(X_train_scaled, y_train)

        # 6. 파이프라인 객체 생성 및 저장
        ml_pipeline = {
            'scaler': scaler,
            'model': model,
            'features': features # ML Core가 예측 시 피처 순서를 맞추기 위해 필수
        }

        os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)
        joblib.dump(ml_pipeline, MODEL_PATH)

        print("-" * 30)
        print(f"✅ 프로토타입 ML 모델 생성 완료!")
        print(f"파일이 '{MODEL_PATH}' 경로에 저장되었습니다.")

    except FileNotFoundError:
        print(f"오류: {DATA_PATH}를 찾을 수 없습니다.")
        print("이전 1단계에서 data/ 폴더로 파일을 올바르게 이동했는지 확인하세요.")
    except Exception as e:
        print(f"예상치 못한 오류 발생: {e}")

환경 변수 확인...
CSV 데이터 로딩 중...
TQ=72인 데이터만 필터링하여 훈련 세트 구성 중...
총 17239건의 데이터로 모델 학습 시작...
학습될 불량(Fail) 샘플 수: 85 (8건이어야 함)
------------------------------
✅ 프로토타입 ML 모델 생성 완료!
파일이 '../models/ml_core_pipeline.joblib' 경로에 저장되었습니다.


In [4]:
# 1. ML 파이프라인(스케일러 + 모델) 로드
try:
    pipeline_data = joblib.load(MODEL_PATH)
    scaler = pipeline_data['scaler']
    model = pipeline_data['model']
    features_order = pipeline_data['features']
    print(f"'{MODEL_PATH}' 로드 성공.")
except FileNotFoundError:
    print(f"오류: '{MODEL_PATH}'를 찾을 수 없습니다. '셀 3'을 먼저 실행하세요.")

# 2. 데이터 로드 (시뮬레이션용)
try:
    df_full = pd.read_csv(DATA_PATH)
    df_full.dropna(subset=['passorfail'], inplace=True)
    print(f"'{DATA_PATH}' 로드 성공. (총 {len(df_full)}개 데이터)")
except FileNotFoundError:
    print(f"오류: '{DATA_PATH}'를 찾을 수 없습니다.")

'../models/ml_core_pipeline.joblib' 로드 성공.
'../data/kamp_main_data.csv' 로드 성공. (총 17264개 데이터)


In [6]:
# --- 1. (TL) 좌측 상단: 꺾은선 그래프 ---
line_chart_fig = go.FigureWidget(
    layout=go.Layout(title="주요 파라미터 실시간 모니터링", height=400)
)
line_chart_fig.add_scatter(y=[], name="EX1.Z1_PV", line=dict(color='#0072B2'))
line_chart_fig.add_scatter(y=[], name="EX1.MELT_P_PV", line=dict(color='#D55E00'))
line_chart_fig.add_scatter(y=[], name="EX1.MD_PV", line=dict(color='#009E73'))

line_chart_fig.add_hline(y=OPTIMAL_VALUES['EX1.Z1_PV'], line=dict(color='#0072B2', dash='dot'))
line_chart_fig.add_hline(y=OPTIMAL_VALUES['EX1.MELT_P_PV'], line=dict(color='#D55E00', dash='dot'))
line_chart_fig.add_hline(y=OPTIMAL_VALUES['EX1.MD_PV'], line=dict(color='#009E73', dash='dot'))


# --- 2. (TR) 우측 상단: 실시간 상태 ---
label_part_no = widgets.Label("0", layout=Layout(width='100%'))
label_yield_rate = widgets.Label("0.00 %", layout=Layout(width='100%'))
label_status = widgets.Label("WAITING...", 
                             style={'font_size': '40px', 'font_weight': 'bold', 'text_align': 'center'},
                             layout=Layout(width='100%', height='300px', border='1px solid #DDDDDD'))

panel_tr = VBox([
    HBox([widgets.Label("Part No.:"), label_part_no]),
    HBox([widgets.Label("Yield Rate:"), label_yield_rate]),
    label_status
])

# --- 3. (BL) 좌측 하단: 실시간 테이블 ---
table_output = widgets.Output(layout=Layout(height='300px', border='1px solid #DDDDDD', overflow_y='auto'))

# --- 4. (BR) 우측 하단: 막대 그래프 ---
bar_chart_fig = go.FigureWidget(
    data=[go.Bar(x=['GOOD', 'BAD'], y=[0, 0], marker_color=['#009E73', '#D55E00'])],
    layout=go.Layout(title="양품/불량 집계", height=300, yaxis_title="Count")
)
label_br_total = widgets.Label("0")
label_br_yield = widgets.Label("0.00 %")

panel_br = VBox([
    bar_chart_fig,
    HBox([widgets.Label("Total Count:"), label_br_total]),
    HBox([widgets.Label("Yield Rate:"), label_br_yield])
])

print("UI 위젯 생성 완료.")

UI 위젯 생성 완료.


In [7]:
# UI 업데이트 함수 (워커 스레드가 호출)
def update_ui(data_packet):
    # (TR) 실시간 상태 업데이트
    label_part_no.value = str(data_packet['total_count'])
    label_yield_rate.value = f"{data_packet['yield_rate']:.2f} %"
    label_status.value = data_packet['status']
    if data_packet['status'] == "GOOD":
        label_status.style.color = '#2E7D32'
        label_status.layout.border = '2px solid #C8E6C9'
    else:
        label_status.style.color = '#C62828'
        label_status.layout.border = '2px solid #FFCDD2'

    # (TL) 꺾은선 그래프 업데이트
    time_points.append(data_packet['index'])
    param_z1_data.append(data_packet['params']['EX1.Z1_PV'])
    param_melt_p_data.append(data_packet['params']['EX1.MELT_P_PV'])
    param_md_data.append(data_packet['params']['EX1.MD_PV'])
    
    if len(time_points) > MAX_GRAPH_POINTS:
        time_points.pop(0); param_z1_data.pop(0); param_melt_p_data.pop(0); param_md_data.pop(0)
    
    with line_chart_fig.batch_update():
        line_chart_fig.data[0].x = time_points
        line_chart_fig.data[0].y = param_z1_data
        line_chart_fig.data[1].x = time_points
        line_chart_fig.data[1].y = param_melt_p_data
        line_chart_fig.data[2].x = time_points
        line_chart_fig.data[2].y = param_md_data

    # (BL) 실시간 테이블 업데이트
    log_data.insert(0, {
        "No.": data_packet['index'],
        "Z1 편차": f"{data_packet['deviations']['Z1']:.2f}",
        "MELT_P 편차": f"{data_packet['deviations']['MELT_P']:.2f}",
        "MD 편차": f"{data_packet['deviations']['MD']:.2f}",
        "Status": data_packet['status']
    })
    if len(log_data) > 100: log_data.pop()
        
    with table_output:
        clear_output(wait=True)
        display(pd.DataFrame(log_data).head(20))
        
    # (BR) 막대 그래프 업데이트
    with bar_chart_fig.batch_update():
        bar_chart_fig.data[0].y = [data_packet['good_count'], data_packet['bad_count']]
    label_br_total.value = str(data_packet['total_count'])
    label_br_yield.value = f"{data_packet['yield_rate']:.2f} %"

# --- ML Core (백엔드 로직) ---
def ml_core_worker():
    good_count = 0
    bad_count = 0
    
    for index, row in df_full.iterrows():
        total_count = index + 1
        
        # (핵심 로직 1) 규칙 기반: TQ=0이면 100% 불량
        if row['EX1.MD_TQ'] == 0:
            prediction = 1
        else:
            # (핵심 로직 2) ML 모델 예측 (TQ=72인 경우)
            current_features_df = pd.DataFrame([row[features_order]])
            current_features_scaled = scaler.transform(current_features_df)
            prediction = model.predict(current_features_scaled)[0]
        
        # 편차 계산
        dev_z1 = row['EX1.Z1_PV'] - OPTIMAL_VALUES['EX1.Z1_PV']
        dev_melt_p = row['EX1.MELT_P_PV'] - OPTIMAL_VALUES['EX1.MELT_P_PV']
        dev_md = row['EX1.MD_PV'] - OPTIMAL_VALUES['EX1.MD_PV']

        # Count 계산
        if prediction == 0:
            good_count += 1; status = "GOOD"
        else:
            bad_count += 1; status = "BAD"
        yield_rate = (good_count / total_count) * 100

        data_packet = {
            'index': total_count, 'status': status, 'prediction': prediction,
            'yield_rate': yield_rate, 'good_count': good_count, 'bad_count': bad_count,
            'total_count': total_count,
            'params': {
                'EX1.Z1_PV': row['EX1.Z1_PV'],
                'EX1.MELT_P_PV': row['EX1.MELT_P_PV'],
                'EX1.MD_PV': row['EX1.MD_PV']
            },
            'deviations': {'Z1': dev_z1, 'MELT_P': dev_melt_p, 'MD': dev_md}
        }
        
        update_ui(data_packet)
        time.sleep(0.05) # 시뮬레이션 속도
        
    print("ML Core Worker: 시뮬레이션 완료.")

print("ML Core 로직 및 UI 업데이트 함수 정의 완료.")

ML Core 로직 및 UI 업데이트 함수 정의 완료.


In [ ]:
# --- (요청사항 3) 4분할 레이아웃 조립 ---
print("UI 레이아웃 조립 중...")

# (수정) HBox가 자식들의 레이아웃을 직접 제어하도록 'grid_template_columns'를 사용합니다.
# 이렇게 하면 자식 위젯(line_chart_fig)의 .layout.width를 직접 건드릴 필요가 없습니다.

top_row = HBox(
    [line_chart_fig, panel_tr], 
    layout=Layout(width='100%', grid_template_columns='60% 40%')
)

bottom_row = HBox(
    [table_output, panel_br], 
    layout=Layout(width='100%', grid_template_columns='60% 40%')
)

# (수정) 전체 UI의 높이를 지정하여 레이아웃이 깨지지 않게 합니다.
# (그래프 400px + 테이블 300px + 여백)
final_ui = VBox(
    [top_row, bottom_row],
    layout=Layout(height='750px')
)


print("UI 레이아웃 조립 완료. 시뮬레이션을 시작합니다...")

# --- ML Core 스레드 생성 및 시작 ---
# (참고: 커널 재시작(Restart) 없이 이 셀을 다시 실행하면 스레드가 중복 실행될 수 있습니다.)
worker_thread = threading.Thread(target=ml_core_worker, daemon=True)
worker_thread.start()

# --- 최종 UI 표시 ---
display(final_ui)

UI 레이아웃 조립 중...
UI 레이아웃 조립 완료. 시뮬레이션을 시작합니다...


    'data': [{'line': {'color': '#0072B2'},
              'name':…